In [1]:
import numpy as np
import scipy.optimize as opt
from scipy.optimize import LinearConstraint, Bounds

Huo and Fu basically interpolate between UCB1 and CVaR, both of which are implemented here to some extent. It's probably worth adding two more benchmarks (UCB1 and CVaR), since I've already written some code for it...

In [ ]:
# globally accessible constants or data

# number of assets
K = 10

# something about finance
delta = 5

# confidence level
gamma = 0.95 # see page 7 below Theorem 2.2

# time horizon
time_horizon = 0 # not sure if this is actually a variable??

# the number of times an asset has been picked; to be updated as the learner runs
times_selected = np.zeros(K) # add to the corresponding entry whenever an asset is selected

# shapes of matrices need to be fixed
H_matrix = np.zeros(shape=(K, K)) # historical returns
P_matrix = np.zeros(shape=(K, K)) # TODO: definition; price matrix?? ("number of assets" x "time horizon")?
R_matrix = np.zeros(shape=(K, K)) # TODO: definition
for t in range(time_horizon): # see definition of R above equation 2.1
    R_matrix[t] = P_matrix[t + 1] / P_matrix[t]
R_matrix = np.log(R_matrix)

In [2]:
def I(t):
    """
    Definition: Equation 2.1; follows the UCB1 algorithm.
    t: the time

    """
    if t < K: # sharp inequality because assets are 0, 1, ..., K - 1
        return t
    else:
        R_bar = R_matrix.mean(axis=1) # take the mean of the returns matrix across the time axis
        return np.argmax([R_bar[i] + np.sqrt((2 * np.log(t) / times_selected[i] * (t - 1))) for i in range(K)])

In [ ]:
def F(gamma, u, alpha, t, H_matrix, R_matrix, delta=delta):
    """
    Definition: Equation 2.3; estimates the conditional value-at-risk.
    H_matrix: the s-th column of H_matrix (K rows, \delta columns) is the historical returns of the s-th asset
    R_matrix: the s-th column of R_matrix (K rows, t - 1 columns) is the trial of returns observed so far of the s-th asset
    """
    sum1 = np.sum([np.max(-np.dot(H_matrix[s], u) - alpha, 0) for s in range(delta)])
    sum2 = np.sum([np.max(-np.dot(R_matrix[s], u) - alpha, 0) for s in range(t - 1)])
    return alpha + (sum1 + sum2) / ((delta + t - 1) * (1 - gamma))

In [ ]:
def omega_C(t):
    """
    Definition: Equation 2.4
    Computes a risk-aware portfolio according to Equation 2.3.
    t: the time
    """
    # the objective function at iteration t
    def F_t(args):
        u, alpha = args[0], args[1]
        return F(gamma, u, alpha, t, H_matrix, R_matrix)

    # constraining u to be a probability distribution over 1, 2, ..., K
    sum_to_one = LinearConstraint(np.ones(K), [0], [1])
    positivity_bounds = Bounds(np.zeros(K), np.ones(K))

    # scipy.optimize.minimize
    x0 = np.ones(K + 1)
    x0[:K] /= K # first K components should sum to 1; no idea what alpha (last component) should represent
    approx_min = opt.minimize(
        F_t, x0=x0,
        constraints=[sum_to_one],
        bounds=positivity_bounds
    )
    return approx_min

In [4]:
def sequential_selection_algo(K, lambda_, gamma=gamma, time_horizon=time_horizon):
    """
    K: number of assets
    gamma: confidence level
    lambda_: risk preference
    """
    returns = []
    for t in range(time_horizon):
        # Equation. 2.2: compute omega_M(t)
        omega_M = np.zeros(K)
        omega_M[I(t)] = 1
        
        # Equation 2.4: compute the risk aware portfolio
        risk_aware_portfolio = omega_C(t)

        # Equation 2.5: compute convex combination of UCB1 portfolio and risk aware portfolio
        omega_star = lambda_ * omega_M + (1 - lambda_) * risk_aware_portfolio

        # "Receive portfolio reward"
        returns_by_t = np.dot(omega_star, R_matrix[:, t])
        returns.append(returns_by_t)
    return returns # idk what to return; there might be a formula somewhere